Rental data cleaning and exploration for the listings scraped in Tartu and Tallinn
In this notebook we look over the scraped data from kv.ee and discard any unimportant, unhelpful or unneccesary data

In [26]:
#Importing required libraries

import pandas as pd
import json
import numpy as np
import os
from pathlib import Path

#Display settings for better data viewing

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)


data_folder = Path('../../Data_11.04.25')


Listing all the files in our specific data folder

In [27]:
city_files = {
    'tallinn': data_folder / 'scraped_listings_tln.json',
    'tartu': data_folder / 'scraped_listings_trt.json'
}

Creating cleaning functions for the data

First extracting price from format price / area

In [28]:

def extract_price(price_str):
    """Extract monthly rent from '450 € 6.79 €/m²' format"""
    if pd.isna(price_str):
        return np.nan
    try:
        clean_str = str(price_str).replace(' ', '').split('€')[0]
        return float(clean_str)
    except:
        return np.nan

Extracting area from similar format

In [29]:
def extract_sqm(area_str):
    """Extract numeric area from '66.3 m²' format"""
    if pd.isna(area_str):
        return np.nan
    try:
        clean_str = str(area_str).replace(' ', '').replace('m²', '')
        return float(clean_str)
    except:
        return np.nan

Extracting the floor / floors from formatted data

In [30]:
def extract_floor(floor_str):
    """Extract floor and total floors from '3/9' format"""
    if pd.isna(floor_str):
        return np.nan, np.nan
    try:
        parts = floor_str.split('/')
        floor = int(parts[0].strip())
        total = int(parts[1].strip())
        return floor, total
    except:
        return np.nan, np.nan

Cleaning each city data with a function

In [ ]:
def clean_city_data(file_path, city_name):
    """Clean data for a single city"""
    print(f"\n{'='*70}")
    print(f"PROCESSING: {city_name.upper()}")
    print(f"{'='*70}")
    
    # Loading data from json
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            print(f"✓ Loaded {len(data)} records from {file_path.name}")
    except Exception as e:
        print(f"✗ Error loading {file_path.name}: {e}")
        return None
    
    # Converting to DataFrame
    df = pd.DataFrame(data)
    print(f"Initial DataFrame shape: {df.shape}")
    print(f"Initial columns: {list(df.columns)}\n")
    
    # Stripping whitespace and replacing "?" with NaN
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].str.strip()
            df[column] = df[column].replace(['?', ' ?', '? '], np.nan)
            df[column] = df[column].str.replace('\xa0', ' ', regex=False)
    
    # Extracting clean numeric values using our functions
    df['price_clean'] = df['price'].apply(extract_price)
    df['area_sqm'] = df['Üldpind'].apply(extract_sqm)
    df[['floor', 'total_floors']] = df['Korrus/Korruseid'].apply(
        lambda x: pd.Series(extract_floor(x))
    )
    df['rooms'] = pd.to_numeric(df['Tube'], errors='coerce')
    
    # Cleaning the build year (no values under 1800 or over 2025)
    df['build_year'] = pd.to_numeric(df['Ehitusaasta'], errors='coerce')
    df.loc[df['build_year'] < 1800, 'build_year'] = np.nan
    df.loc[df['build_year'] > 2025, 'build_year'] = np.nan
    
    # Validating coordinates - going to numeric
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    
    # COLUMN SELECTION BASED ON MISSING DATA THRESHOLD
    print(f"{'='*70}")
    print("COLUMN EVALUATION - Missing Data Analysis (Threshold: 20%)")
    print(f"{'='*70}\n")
    
    missing_threshold = 20  # Removing columns with >20% missing data
    
    # Define columns to exclude or always keep
    exclude_cols = [
        'price', 'Üldpind', 'Korrus/Korruseid', 'Ehitusaasta', 'Tube'
    ]
    essential_cols = [
        'id', 'url', 'latitude', 'longitude'
    ]
    
    # Evaluating all columns
    columns_to_keep = []
    columns_removed = []
    
    for col in df.columns:
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / len(df)) * 100
        
        # Skip if this is a raw column we're replacing with cleaned version
        if col in exclude_cols:
            columns_removed.append(col)
            print(f"✗ REMOVE {col:30s} | {missing_pct:6.2f}% missing | [REPLACED BY CLEANED VERSION]")
        elif col in essential_cols:
            columns_to_keep.append(col)
            print(f"✓ KEEP   {col:30s} | {missing_pct:6.2f}% missing | [ESSENTIAL]")
        elif missing_pct <= missing_threshold:
            columns_to_keep.append(col)
            print(f"✓ KEEP   {col:30s} | {missing_pct:6.2f}% missing")
        else:
            columns_removed.append(col)
            print(f"✗ REMOVE {col:30s} | {missing_pct:6.2f}% missing | [EXCEEDS THRESHOLD]")
    
    print(f"\n{'='*70}")
    print(f"SUMMARY: Kept {len(columns_to_keep)} columns, Removed {len(columns_removed)} columns")
    print(f"{'='*70}")
    
    if columns_removed:
        print(f"\nRemoved columns: {', '.join(columns_removed)}")
    
    # Create filtered dataframe
    df_filtered = df[columns_to_keep].copy()
    
    # Map to clean English names
    column_mapping = {
        'id': 'id',
        'url': 'url',
        'price': 'price_raw',
        'latitude': 'latitude',
        'longitude': 'longitude',
        'Üürida korter': 'rental_ad',
        'Tube': 'rooms_raw',
        'Üldpind': 'area_raw',
        'Korrus/Korruseid': 'floor_raw',
        'Ehitusaasta': 'build_year_raw',
        'Seisukord': 'condition',
        'Korruseid': 'total_floors_alt',
        'Magamistube': 'bedrooms',
        'Energiamärgis': 'energy_label',
        'Omandivorm': 'ownership_type',
        'price_clean': 'price',
        'area_sqm': 'area_sqm',
        'rooms': 'rooms',
        'floor': 'floor',
        'total_floors': 'total_floors',
        'build_year': 'build_year'
    }
    
    # Rename columns
    new_column_names = []
    for col in df_filtered.columns:
        new_name = column_mapping.get(col, col)
        new_column_names.append(new_name)
    
    df_filtered.columns = new_column_names
    
    # If we have both price_raw and price, drop price_raw (keep the cleaned version)
    if 'price' in df_filtered.columns and 'price_raw' in df_filtered.columns:
        df_filtered = df_filtered.drop(columns=['price_raw'])
        print("\n✓ Dropped 'price_raw' - keeping cleaned 'price' column")
    
    # Drop rental_ad column if it exists (not needed)
    if 'rental_ad' in df_filtered.columns:
        df_filtered = df_filtered.drop(columns=['rental_ad'])
        print("✓ Dropped 'rental_ad' - not needed for analysis")
    
    # Same for other duplicate raw columns
    duplicate_raw_cols = [col for col in df_filtered.columns if col.endswith('_raw')]
    if duplicate_raw_cols:
        # Only drop raw columns if we have their cleaned versions
        to_drop = []
        for raw_col in duplicate_raw_cols:
            clean_col = raw_col.replace('_raw', '')
            if clean_col in df_filtered.columns:
                to_drop.append(raw_col)
        if to_drop:
            df_filtered = df_filtered.drop(columns=to_drop)
            print(f"✓ Dropped raw columns: {', '.join(to_drop)}")
    
    print(f"\nFinal columns after cleanup:")
    print(f"{list(df_filtered.columns)}\n")
    
    # DATA QUALITY CHECKS
    print(f"{'='*70}")
    print("DATA QUALITY CHECKS")
    print(f"{'='*70}")
    
    # Removing records missing critical fields
    critical_fields = ['price', 'area_sqm', 'rooms']
    before = len(df_filtered)
    df_final = df_filtered.dropna(subset=critical_fields)
    removed_missing = before - len(df_final)
    print(f"\n✓ Removed {removed_missing} records missing critical fields (price, area, rooms)")
    
    # Remove unrealistic prices
    before = len(df_final)
    df_final = df_final[df_final['price'] >= 50]
    removed_price = before - len(df_final)
    print(f"✓ Removed {removed_price} records with price < 50€ (likely errors)")
    
    # Price distribution
    print(f"\nPrice distribution:")
    print(f"  50-100€:    {((df_final['price'] >= 50) & (df_final['price'] < 100)).sum():4d} listings")
    print(f"  100-500€:   {((df_final['price'] >= 100) & (df_final['price'] < 500)).sum():4d} listings")
    print(f"  500-1000€:  {((df_final['price'] >= 500) & (df_final['price'] < 1000)).sum():4d} listings")
    print(f"  > 1000€:    {(df_final['price'] >= 1000).sum():4d} listings")
    
    # Area distribution
    print(f"\nArea distribution:")
    print(f"  < 15 m²:    {(df_final['area_sqm'] < 15).sum():4d} listings")
    print(f"  15-50 m²:   {((df_final['area_sqm'] >= 15) & (df_final['area_sqm'] < 50)).sum():4d} listings")
    print(f"  50-100 m²:  {((df_final['area_sqm'] >= 50) & (df_final['area_sqm'] < 100)).sum():4d} listings")
    print(f"  > 100 m²:   {(df_final['area_sqm'] >= 100).sum():4d} listings")
    
    print(f"\n{'='*70}")
    print(f"FINAL {city_name.upper()} DATASET: {len(df_final)} rows × {len(df_final.columns)} columns")
    print(f"{'='*70}")
    
    print(f"\nMissing values in final dataset:")
    missing_summary = df_final.isnull().sum()
    missing_summary = missing_summary[missing_summary > 0]
    if len(missing_summary) > 0:
        for col, count in missing_summary.items():
            pct = (count / len(df_final)) * 100
            print(f"  {col:20s}: {count:4d} ({pct:.1f}%)")
    else:
        print("  No missing values in critical columns!")
    
    return df_final

Saving cleaned dataframes to new files

In [32]:
# Process each city
output_folder = Path('../../Cleaned_csvs')
output_folder.mkdir(parents=True, exist_ok=True)

print("\n" + "="*70)
print("RENTAL DATA CLEANING - TARTU & TALLINN")
print("="*70)

results = {}
for city_name, file_path in city_files.items():
    df_city = clean_city_data(file_path, city_name)
    
    if df_city is not None:
        output_path = output_folder / f'listings_cleaned_{city_name}.csv'
        df_city.to_csv(output_path, index=False, encoding='utf-8', sep=";")
        print(f"\n✓ Saved to: {output_path}")
        results[city_name] = len(df_city)

print(f"\n{'='*70}")
print("PROCESSING COMPLETE")
print(f"{'='*70}")
print(f"\nFinal dataset sizes:")
for city, count in results.items():
    print(f"  {city.capitalize():10s}: {count:4d} listings")
print(f"  {'Total':10s}: {sum(results.values()):4d} listings")


RENTAL DATA CLEANING - TARTU & TALLINN

PROCESSING: TALLINN
✓ Loaded 1359 records from scraped_listings_tln.json
Initial DataFrame shape: (1359, 20)
Initial columns: ['id', 'url', 'price', 'latitude', 'longitude', 'Üürida korter', 'Tube', 'Üldpind', 'Korrus/Korruseid', 'Ehitusaasta', 'Seisukord', 'Korruseid', 'Magamistube', 'Energiamärgis', 'Omandivorm', 'Ettemaks', 'Kulud suvel/talvel', 'Katastrinumber', 'Üürida korter (Broneeritud)', 'Registriosa number']

COLUMN EVALUATION - Missing Data Analysis (Threshold: 20%)



NameError: name 'exclude_cols' is not defined